# 🤖 AI YouTube Shorts — Google Colab Quickstart

**Fully automated 11-step pipeline**: Topic → Story → SEO → 5 Scene Images → TTS → SadTalker Avatar → Subtitles → Slideshow → YouTube Upload

📦 **Repository**: [github.com/tamilarasu18/ai-yt-automation](https://github.com/tamilarasu18/ai-yt-automation)

---

## ☀️ Daily Routine (After One-Time Setup)

```
1. Open this Colab notebook         → 5 seconds (bookmark it!)
2. Click "Runtime → Run all"        → 1 click
3. Wait ~15 minutes                 → go drink coffee ☕
4. Telegram notification pops       → "Video uploaded! ✅"
5. Close Colab                      → done for the day
```

> **1 click. 15 minutes. Video live on YouTube.**

### ⚡ Smart Caching
- **First run**: Downloads all models (~6 GB) → caches to Drive
- **Future runs**: Restores from Drive → **saves ~10 min**

---

## 📋 Before You Start (One-Time Setup)

1. **Runtime**: Go to **Runtime → Change runtime type → T4 GPU**
2. **HuggingFace token** (free): Get from https://huggingface.co/settings/tokens
3. **Google Drive** must have this structure:

```
MyDrive/
└── ai-youtube-automation/
    ├── config.json              ← Your settings (includes HF token)
    ├── client_secret.json       ← YouTube OAuth client
    ├── service_account.json     ← Google Sheets auth
    └── videos/                  ← Output videos saved here
```

4. **Google Sheet** must be shared with your service account email

---
## 🔧 Step 1 — Mount Google Drive

In [ ]:
# ═══════════════════════════════════════════════════════════════
# STEP 1: Mount Google Drive
# ═══════════════════════════════════════════════════════════════

from google.colab import drive
drive.mount('/content/drive')

import os

DRIVE_BASE = '/content/drive/MyDrive/ai-youtube-automation'
DRIVE_CACHE = f'{DRIVE_BASE}/models'

# Verify credentials
print('📁 Checking Google Drive...\n')
all_ok = True
for name in ['config.json', 'service_account.json', 'client_secret.json']:
    path = f'{DRIVE_BASE}/{name}'
    exists = os.path.exists(path)
    print(f'  {"✅" if exists else "❌"} {name}')
    if not exists: all_ok = False

cache_exists = os.path.exists(f'{DRIVE_CACHE}/ollama')
print(f'\n  {"✅" if cache_exists else "⏳"} Model cache: {"Found (fast restore!)" if cache_exists else "Not found (first run will download)"}')

if all_ok:
    print('\n✅ Drive ready!')
else:
    print('\n⚠️  Missing files! Upload them to Drive/ai-youtube-automation/')

---
## 📦 Step 2 — Clone Repo & Install Packages

In [ ]:
# ═══════════════════════════════════════════════════════════════
# STEP 2: Clone repo + Install packages (~3 min, always needed)
# ═══════════════════════════════════════════════════════════════

import os

if not os.path.exists('/content/ai-yt-automation'):
    !git clone https://github.com/tamilarasu18/ai-yt-automation.git /content/ai-yt-automation
else:
    !cd /content/ai-yt-automation && git pull

os.chdir('/content/ai-yt-automation')

# System deps (zstd is required by Ollama installer)
!apt-get update -qq && apt-get install -y -qq ffmpeg espeak-ng zstd > /dev/null 2>&1

# Python deps (always reinstalled — Colab resets pip on disconnect)
!pip install -e ".[dev,api,kokoro]" -q 2>/dev/null
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 -q 2>/dev/null
!pip install diffusers transformers accelerate safetensors -q 2>/dev/null
!pip install moviepy==1.0.3 -q 2>/dev/null
!pip install gspread google-auth google-auth-oauthlib google-api-python-client -q 2>/dev/null
!pip install python-telegram-bot==20.7 openai-whisper -q 2>/dev/null

print('\n✅ Packages installed!')

---
## 🤖 Step 3 — Setup Ollama + SadTalker (with Drive Cache)

**First run**: Downloads models → saves to Drive (~6 GB, ~10 min)

**Future runs**: Restores from Drive (~1 min) ⚡

In [ ]:
# ═══════════════════════════════════════════════════════════════
# STEP 3: Setup Ollama + SadTalker with Drive model caching
# ═══════════════════════════════════════════════════════════════

import os, subprocess, time, shutil

DRIVE_BASE = '/content/drive/MyDrive/ai-youtube-automation'
DRIVE_CACHE = f'{DRIVE_BASE}/models'

# ═══════════════════════════════════════════════════════════════
# OLLAMA SETUP
# ═══════════════════════════════════════════════════════════════
print('🤖 Setting up Ollama...')

# Install Ollama (zstd must be installed first — done in Step 2)
!curl -fsSL https://ollama.com/install.sh | sh

# Restore Ollama models from Drive cache (if available)
ollama_cache = f'{DRIVE_CACHE}/ollama'
ollama_local = os.path.expanduser('~/.ollama/models')

if os.path.exists(ollama_cache) and os.listdir(ollama_cache):
    print('⚡ Restoring Ollama model from Drive cache...')
    os.makedirs(ollama_local, exist_ok=True)
    !cp -r {ollama_cache}/* {ollama_local}/ 2>/dev/null
    print('✅ Ollama model restored from cache!')
    need_pull = False
else:
    print('⏳ No cache found — will download model')
    need_pull = True

# Start Ollama server (with retry if binary not found)
try:
    subprocess.Popen(['ollama', 'serve'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, start_new_session=True)
except FileNotFoundError:
    print('⚠️ Ollama binary not in PATH, trying /usr/local/bin/ollama...')
    subprocess.Popen(['/usr/local/bin/ollama', 'serve'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, start_new_session=True)
time.sleep(5)

# Verify server is running
import urllib.request
server_ok = False
for i in range(5):
    try:
        urllib.request.urlopen('http://localhost:11434/api/tags', timeout=3)
        print('✅ Ollama server running!')
        server_ok = True
        break
    except Exception:
        time.sleep(2)
        print(f'   Retrying... ({(i+1)*2}s)')

if not server_ok:
    print('❌ Ollama server failed to start. Try restarting the runtime.')

# Pull model if not cached
if need_pull and server_ok:
    !ollama pull gemma3:4b
    # Cache to Drive for next time
    print('💾 Caching Ollama model to Drive...')
    os.makedirs(ollama_cache, exist_ok=True)
    !cp -r {ollama_local}/* {ollama_cache}/ 2>/dev/null
    print('✅ Ollama model cached to Drive!')
elif server_ok:
    !ollama list

# ═══════════════════════════════════════════════════════════════
# SADTALKER SETUP
# ═══════════════════════════════════════════════════════════════
print('\n🗣️ Setting up SadTalker...')

sadtalker_cache = f'{DRIVE_CACHE}/sadtalker'
sadtalker_local = '/content/SadTalker'

if not os.path.exists(sadtalker_local):
    !git clone -q https://github.com/OpenTalker/SadTalker.git {sadtalker_local}

# Patch for numpy 2.0
!sed -i 's/np.VisibleDeprecationWarning/DeprecationWarning/g' {sadtalker_local}/src/face3d/util/preprocess.py 2>/dev/null; true

# Install SadTalker deps
!pip install -q face-alignment==1.3.5 kornia==0.6.12 yacs facexlib gfpgan basicsr dlib 2>/dev/null
!pip install -q -r {sadtalker_local}/requirements.txt 2>/dev/null; true

# Restore or download SadTalker checkpoints
checkpoints_dir = f'{sadtalker_local}/checkpoints'
if os.path.exists(sadtalker_cache) and os.listdir(sadtalker_cache):
    print('⚡ Restoring SadTalker models from Drive cache...')
    os.makedirs(checkpoints_dir, exist_ok=True)
    !cp -r {sadtalker_cache}/* {checkpoints_dir}/ 2>/dev/null
    gfpgan_cache = f'{DRIVE_CACHE}/gfpgan'
    if os.path.exists(gfpgan_cache):
        !cp -r {gfpgan_cache}/* {sadtalker_local}/gfpgan/ 2>/dev/null; true
    print('✅ SadTalker models restored from cache!')
else:
    print('⏳ Downloading SadTalker models...')
    !cd {sadtalker_local} && bash scripts/download_models.sh 2>/dev/null; true
    print('💾 Caching SadTalker models to Drive...')
    os.makedirs(sadtalker_cache, exist_ok=True)
    !cp -r {checkpoints_dir}/* {sadtalker_cache}/ 2>/dev/null
    gfpgan_cache = f'{DRIVE_CACHE}/gfpgan'
    gfpgan_local = f'{sadtalker_local}/gfpgan'
    if os.path.exists(gfpgan_local):
        os.makedirs(gfpgan_cache, exist_ok=True)
        !cp -r {gfpgan_local}/* {gfpgan_cache}/ 2>/dev/null; true
    print('✅ SadTalker models cached to Drive!')

# ═══════════════════════════════════════════════════════════════
# STABLE DIFFUSION CACHE (HuggingFace)
# ═══════════════════════════════════════════════════════════════
print('\n🖼️ Setting up Stable Diffusion cache...')

hf_cache_drive = f'{DRIVE_CACHE}/huggingface'
hf_cache_local = os.path.expanduser('~/.cache/huggingface')

if os.path.exists(hf_cache_drive) and os.listdir(hf_cache_drive):
    print('⚡ Restoring HuggingFace models from Drive cache...')
    os.makedirs(hf_cache_local, exist_ok=True)
    !cp -r {hf_cache_drive}/* {hf_cache_local}/ 2>/dev/null
    print('✅ SD model restored from cache!')
else:
    print('⏳ SD model will download on first pipeline run')
    print('   (will be cached to Drive automatically after)')

os.chdir('/content/ai-yt-automation')
print('\n' + '='*50)
print('✅ All models ready!')

---
## ⚙️ Step 4 — Load Config & Set Environment

> ⚠️ **HuggingFace Token Required**: Stable Diffusion models now require authentication.
> Get a free token at https://huggingface.co/settings/tokens and add `"hf_token"` to your `config.json`.

In [ ]:
# ═══════════════════════════════════════════════════════════════
# STEP 4: Load config from Drive + set environment variables
# ═══════════════════════════════════════════════════════════════

import json, os, shutil

DRIVE_BASE = '/content/drive/MyDrive/ai-youtube-automation'
WORK_DIR = '/content/ai-yt-automation'

# Copy credential files from Drive
for fname in ['service_account.json', 'client_secret.json']:
    src = f'{DRIVE_BASE}/{fname}'
    dst = f'{WORK_DIR}/{fname}'
    if os.path.exists(src):
        shutil.copy(src, dst)
        print(f'  ✅ {fname}')
    else:
        print(f'  ⚠️  {fname} not found')

# Load config.json
config = {}
config_path = f'{DRIVE_BASE}/config.json'
if os.path.exists(config_path):
    with open(config_path) as f:
        config = json.load(f)
    print(f'  ✅ config.json loaded')

# Map config keys → env vars
env_map = {
    'google_sheet_url': 'GOOGLE_SHEET_URL',
    'google_sheet_name': 'GOOGLE_SHEET_NAME',
    'youtube_client_id': 'YOUTUBE_CLIENT_ID',
    'youtube_client_secret': 'YOUTUBE_CLIENT_SECRET',
    'youtube_refresh_token': 'YOUTUBE_REFRESH_TOKEN',
    'telegram_bot_token': 'TELEGRAM_BOT_TOKEN',
    'telegram_chat_id': 'TELEGRAM_CHAT_ID',
    'video_privacy': 'VIDEO_PRIVACY',
    'auto_upload_youtube': 'AUTO_UPLOAD_YOUTUBE',
}
for k, v in env_map.items():
    val = config.get(k, '')
    if val: os.environ[v] = str(val)

# ── HuggingFace Authentication ──
# Required for Stable Diffusion model download
hf_token = config.get('hf_token', '')
if hf_token:
    os.environ['HF_TOKEN'] = hf_token
    os.environ['HUGGING_FACE_HUB_TOKEN'] = hf_token
    print(f'  ✅ HuggingFace token set')
else:
    print('  ⚠️  No hf_token in config.json — SD model download may fail!')
    print('     Get a free token at: https://huggingface.co/settings/tokens')
    print('     Add to config.json: "hf_token": "hf_xxxxxxxxxxxx"')

# Fixed Colab paths
os.environ['GOOGLE_SERVICE_ACCOUNT_FILE'] = f'{WORK_DIR}/service_account.json'
os.environ['WORK_DIR'] = WORK_DIR
os.environ['OUTPUT_DIR'] = f'{WORK_DIR}/output'
os.environ['AVATAR_IMAGE_PATH'] = f'{WORK_DIR}/assets/images/avatar.png'
os.environ['SADTALKER_DIR'] = '/content/SadTalker'
os.environ['DRIVE_OUTPUT_FOLDER'] = config.get('drive_output_folder', f'{DRIVE_BASE}/videos')

# Engine config (optimized for T4)
os.environ['OLLAMA_HOST'] = 'http://localhost:11434'
os.environ['OLLAMA_MODEL'] = 'gemma3:4b'
os.environ['TTS_ENGINE'] = 'edge'
os.environ['IMAGE_ENGINE'] = 'sd'
os.environ['SD_MODEL'] = 'stabilityai/stable-diffusion-2-1'
os.environ['ENABLE_FACE_ENHANCEMENT'] = 'true'
os.environ['WHISPER_MODEL_SIZE'] = 'base'
os.environ['SDXL_INFERENCE_STEPS'] = '20'

# Check avatar exists (should be in the cloned repo)
avatar_path = os.environ['AVATAR_IMAGE_PATH']
if not os.path.exists(avatar_path):
    # Fallback: try to copy from Drive
    for ext in ['png', 'webp', 'jpg']:
        drive_avatar = f'{DRIVE_BASE}/avatar.{ext}'
        if os.path.exists(drive_avatar):
            os.makedirs(os.path.dirname(avatar_path), exist_ok=True)
            shutil.copy(drive_avatar, avatar_path)
            print(f'  ✅ Avatar copied from Drive ({ext})')
            break
    else:
        print('  ⚠️  Avatar not found! Upload avatar.png to Drive/ai-youtube-automation/')
else:
    print(f'  ✅ Avatar found in repo')

os.makedirs(os.environ['OUTPUT_DIR'], exist_ok=True)
os.makedirs(os.environ['DRIVE_OUTPUT_FOLDER'], exist_ok=True)

print(f'\n📋 Config: TTS={os.environ["TTS_ENGINE"]} | Images={os.environ["SD_MODEL"].split("/")[-1]} | LLM={os.environ["OLLAMA_MODEL"]}')
print(f'   HF Auth: {"✅" if hf_token else "❌ Missing"}')
print('✅ Environment ready!')

---
## ✅ Step 5 — Verify Everything

In [ ]:
# ═══════════════════════════════════════════════════════════════
# STEP 5: System check
# ═══════════════════════════════════════════════════════════════

import os, subprocess
os.chdir('/content/ai-yt-automation')

print('🔍 System Check\n')
checks = []

# GPU
import torch
gpu = torch.cuda.is_available()
checks.append(('GPU', gpu, torch.cuda.get_device_name(0) if gpu else 'No GPU'))

# Ollama
try:
    r = subprocess.run(['ollama', 'list'], capture_output=True, text=True, timeout=5)
    checks.append(('Ollama', r.returncode == 0, 'Running'))
except: checks.append(('Ollama', False, 'Not running'))

# FFmpeg
try:
    r = subprocess.run(['ffmpeg', '-version'], capture_output=True, text=True, timeout=5)
    checks.append(('FFmpeg', r.returncode == 0, 'Installed'))
except: checks.append(('FFmpeg', False, 'Missing'))

# Files
for name, path in [
    ('Avatar', os.environ.get('AVATAR_IMAGE_PATH', '')),
    ('Credentials', os.environ.get('GOOGLE_SERVICE_ACCOUNT_FILE', '')),
    ('SadTalker', '/content/SadTalker/checkpoints'),
]:
    checks.append((name, os.path.exists(path), path.split('/')[-1]))

# HuggingFace token
hf_ok = bool(os.environ.get('HF_TOKEN', ''))
checks.append(('HF Token', hf_ok, 'Set' if hf_ok else 'Missing — add hf_token to config.json'))

# Python packages
for pkg_name, imp in [('Edge TTS','edge_tts'),('Diffusers','diffusers'),('Whisper','whisper'),('MoviePy','moviepy')]:
    try: __import__(imp); checks.append((pkg_name, True, '✓'))
    except: checks.append((pkg_name, False, 'Missing'))

# Drive cache
DRIVE_BASE = '/content/drive/MyDrive/ai-youtube-automation'
cache = os.path.exists(f'{DRIVE_BASE}/models/ollama')
checks.append(('Model Cache', cache, 'Cached on Drive' if cache else 'Will cache after first run'))

for name, ok, detail in checks:
    print(f'  {"✅" if ok else "❌"} {name:16s} {detail}')

failed = sum(1 for _, ok, _ in checks if not ok)
print(f'\n{"🚀 Ready!" if failed == 0 else f"⚠️ {failed} issue(s) — fix above"}')

---
## 🧪 Step 6 — Test Run

In [ ]:
# ═══════════════════════════════════════════════════════════════
# STEP 6: Test pipeline (story → 5 images → audio)
# ═══════════════════════════════════════════════════════════════

import os
os.chdir('/content/ai-yt-automation')
!python -m ai_shorts run --mode test

---
## 🎬 Step 7 — Full Pipeline

```
 1. Fetch topic        → Google Sheet
 2. Generate story     → Ollama (randomized styles)
 3. Generate SEO       → title, description, hashtags
 4. Generate 5 prompts → one per scene
 5. Generate 5 images  → Stable Diffusion (T4 GPU)
 6. Generate audio     → Edge TTS
 7. Animate avatar     → SadTalker (talking head)
 8. Generate subtitles → Whisper
 9. Compose slideshow  → MoviePy (5 images + avatar overlay)
10. Upload YouTube     → resumable + scheduler
11. Backup + Notify    → Google Drive + Telegram
```

In [ ]:
# ═══════════════════════════════════════════════════════════════
# STEP 7: Full pipeline 🚀
# ═══════════════════════════════════════════════════════════════

import os
os.chdir('/content/ai-yt-automation')
!python -m ai_shorts run --mode full

---
## 💾 Step 8 — Cache SD Model to Drive (run after first pipeline)

In [ ]:
# ═══════════════════════════════════════════════════════════════
# STEP 8: Cache HuggingFace SD model to Drive (run ONCE)
# ═══════════════════════════════════════════════════════════════

import os

DRIVE_CACHE = '/content/drive/MyDrive/ai-youtube-automation/models'
hf_local = os.path.expanduser('~/.cache/huggingface')
hf_drive = f'{DRIVE_CACHE}/huggingface'

if os.path.exists(hf_local) and not os.path.exists(hf_drive):
    print('💾 Caching HuggingFace SD model to Drive...')
    print('   This may take 2-3 minutes (one-time only)')
    os.makedirs(hf_drive, exist_ok=True)
    !cp -r {hf_local}/* {hf_drive}/ 2>/dev/null
    print('✅ SD model cached to Drive!')
elif os.path.exists(hf_drive):
    print('✅ SD model already cached on Drive')
else:
    print('⚠️  Run the pipeline first to download the SD model')

---
## 📺 Step 9 — Preview & Download

In [ ]:
# ═══════════════════════════════════════════════════════════════
# STEP 9: Preview + download
# ═══════════════════════════════════════════════════════════════

from IPython.display import Video, display, Image
from pathlib import Path
import glob, shutil, os

output_dir = Path('/content/ai-yt-automation/output')
DRIVE_BASE = '/content/drive/MyDrive/ai-youtube-automation'

# Show scene images
scenes = sorted(glob.glob(str(output_dir / '*/scenes/scene_*.png')))
if scenes:
    print(f'🖼️  {len(scenes)} Scene Images:\n')
    for img in scenes:
        display(Image(filename=img, width=250))

# Show video
videos = sorted(glob.glob(str(output_dir / '*/final_video.mp4')))
if videos:
    print(f'\n🎬 Final Video:')
    display(Video(videos[-1], width=360))
    drive_out = os.environ.get('DRIVE_OUTPUT_FOLDER', f'{DRIVE_BASE}/videos')
    os.makedirs(drive_out, exist_ok=True)
    shutil.copy(videos[-1], f'{drive_out}/latest_video.mp4')
    print(f'💾 Saved to: {drive_out}/latest_video.mp4')
else:
    print('⚠️  No video found')

In [ ]:
# Download to your computer
from google.colab import files
import glob
videos = sorted(glob.glob('/content/ai-yt-automation/output/*/final_video.mp4'))
if videos: files.download(videos[-1])

---
## 🔑 One-Time: YouTube OAuth Token

Run only ONCE to generate your YouTube refresh token.

In [ ]:
# ONE-TIME: Generate YouTube refresh token
!pip install google-auth-oauthlib google-api-python-client -q

from google_auth_oauthlib.flow import InstalledAppFlow
import os

DRIVE_BASE = '/content/drive/MyDrive/ai-youtube-automation'
secret = '/content/ai-yt-automation/client_secret.json'
if not os.path.exists(secret):
    secret = f'{DRIVE_BASE}/client_secret.json'

flow = InstalledAppFlow.from_client_secrets_file(
    secret, ['https://www.googleapis.com/auth/youtube.upload']
)
creds = flow.run_local_server(port=0)

print('=' * 50)
print('🔑 Add these to config.json in Drive:')
print('=' * 50)
print(f'"youtube_client_id": "{creds.client_id}",')
print(f'"youtube_client_secret": "{creds.client_secret}",')
print(f'"youtube_refresh_token": "{creds.refresh_token}"')

---

## 📝 config.json Template

```json
{
  "hf_token": "hf_xxxxxxxxxxxxxxxxxxxx",
  "google_sheet_url": "https://docs.google.com/spreadsheets/d/YOUR_SHEET_ID/edit",
  "google_sheet_name": "Sheet1",
  "youtube_client_id": "YOUR_CLIENT_ID.apps.googleusercontent.com",
  "youtube_client_secret": "GOCSPX-your-secret",
  "youtube_refresh_token": "1//your-refresh-token",
  "telegram_bot_token": "1234567890:AAxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
  "telegram_chat_id": "123456789",
  "auto_upload_youtube": true,
  "video_privacy": "public",
  "drive_output_folder": "/content/drive/MyDrive/ai-youtube-automation/videos"
}
```

> ⚠️ **Get your HuggingFace token** (free): https://huggingface.co/settings/tokens

## 🛠️ Troubleshooting

| Issue | Fix |
|-------|-----|
| `CUDA out of memory` | Runtime → Restart runtime → Run all |
| `401 Unauthorized` (SD model) | Add `hf_token` to config.json |
| `Ollama connection refused` | Re-run Step 3 |
| `Avatar not found` | Upload avatar.png to Drive or re-clone repo |
| `SadTalker model not found` | Re-run Step 3 |
| `Edge TTS timeout` | Change to `kokoro` in Step 4 |
| `No topics found` | Add rows with Status=`Pending` in Sheet |
| `YouTube token expired` | Re-run OAuth cell above |

### 📁 Drive Structure After First Run
```
MyDrive/ai-youtube-automation/
├── config.json                ← Must include hf_token!
├── service_account.json
├── client_secret.json
├── models/                    ← Auto-cached (~6 GB)
│   ├── ollama/                ← Gemma 3 4B
│   ├── sadtalker/             ← SadTalker checkpoints
│   ├── gfpgan/                ← Face enhancement
│   └── huggingface/           ← Stable Diffusion 2.1
└── videos/                    ← Output videos
```

---

**Built with Clean Architecture ❤️ — Your AI Employee Working 24/7**